In [71]:
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
from geopy.distance import vincenty
import sys

foursquare_data = pd.read_csv("./data/foursquare-nyc-and-tokyo-check-ins/dataset_TSMC2014_NYC.csv")
climate_data = pd.read_csv("./data/1087533.csv")

/Users/frankyliang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (8,10,11,14,15,20,23,25,40,68,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [72]:
pd.options.display.max_columns = 999
climate_data.HOURLYPrecip = climate_data.HOURLYPrecip.fillna(0.00).replace('T', '0.00').str.replace(r"[a-zA-Z]",'').astype(float)

In [79]:
def closest_station(point, points):
    closest_station = ""
    closest_distance = 50000                                                 
    for station in points:
        cur_dist = vincenty(point, station).kilometers
        if cur_dist < closest_distance:
            closest_station = station_lat_long[station_lat_long['lat_long'] == station]['STATION_NAME'].values[0]
    return closest_station

station_lat_long = climate_data[['STATION_NAME', 'LATITUDE', 'LONGITUDE']].drop_duplicates()
climate_data['lat_long'] = [(x, y) for x,y in zip(climate_data['LATITUDE'], climate_data['LONGITUDE'])]
foursquare_data['lat_long'] = [(x, y) for x,y in zip(foursquare_data['latitude'], foursquare_data['longitude'])]                                       
station_lat_long['lat_long'] = [(x, y) for x,y in zip(station_lat_long['LATITUDE'], station_lat_long['LONGITUDE'])]

foursquare_data['closestStation'] = [closest_station(pt, station_lat_long['lat_long']) for pt in foursquare_data['lat_long']]



In [83]:
climate_data.DATE = pd.to_datetime(climate_data.DATE, utc=True)
climate_data.sort_values('DATE', inplace=True)
foursquare_data.utcTimestamp = pd.to_datetime(foursquare_data.utcTimestamp, utc=True)
foursquare_data.sort_values('utcTimestamp', inplace=True)

0       2012-04-12 00:51:00
22104   2012-04-12 00:51:00
11739   2012-04-12 00:51:00
1       2012-04-12 01:00:00
22105   2012-04-12 01:00:00
2       2012-04-12 01:51:00
22106   2012-04-12 01:51:00
11740   2012-04-12 01:51:00
11741   2012-04-12 02:51:00
22107   2012-04-12 02:51:00
3       2012-04-12 02:51:00
4       2012-04-12 03:51:00
22108   2012-04-12 03:51:00
11742   2012-04-12 03:51:00
11743   2012-04-12 04:00:00
5       2012-04-12 04:00:00
22109   2012-04-12 04:00:00
11744   2012-04-12 04:51:00
6       2012-04-12 04:51:00
22110   2012-04-12 04:51:00
22111   2012-04-12 05:51:00
7       2012-04-12 05:51:00
11745   2012-04-12 05:51:00
22112   2012-04-12 06:51:00
8       2012-04-12 06:51:00
11746   2012-04-12 06:51:00
22113   2012-04-12 07:00:00
9       2012-04-12 07:00:00
22114   2012-04-12 07:51:00
10      2012-04-12 07:51:00
                ...        
11728   2013-02-16 16:51:00
22095   2013-02-16 16:51:00
22096   2013-02-16 17:51:00
11729   2013-02-16 17:51:00
33679   2013-02-16 1

In [6]:
climate_data.DATE.dt.round('5min')
foursquare_data.utcTimestamp = pd.to_datetime(foursquare_data.utcTimestamp).dt.round('5min')

0       2012-04-12 00:50:00
1       2012-04-12 01:00:00
2       2012-04-12 01:50:00
3       2012-04-12 02:50:00
4       2012-04-12 03:50:00
5       2012-04-12 04:00:00
6       2012-04-12 04:50:00
7       2012-04-12 05:50:00
8       2012-04-12 06:50:00
9       2012-04-12 07:00:00
10      2012-04-12 07:50:00
11      2012-04-12 08:50:00
12      2012-04-12 09:50:00
13      2012-04-12 10:00:00
14      2012-04-12 10:50:00
15      2012-04-12 11:50:00
16      2012-04-12 12:50:00
17      2012-04-12 13:00:00
18      2012-04-12 13:50:00
19      2012-04-12 14:50:00
20      2012-04-12 15:50:00
21      2012-04-12 16:00:00
22      2012-04-12 16:50:00
23      2012-04-12 17:50:00
24      2012-04-12 18:50:00
25      2012-04-12 19:00:00
26      2012-04-12 19:50:00
27      2012-04-12 20:50:00
28      2012-04-12 21:50:00
29      2012-04-12 22:00:00
                ...        
33659   2013-02-16 04:50:00
33660   2013-02-16 05:50:00
33661   2013-02-16 05:50:00
33662   2013-02-16 06:50:00
33663   2013-02-16 0

In [102]:
climate_data['roundedDate'] = climate_data.DATE.dt.round('60min')
foursquare_data['roundedDate'] = foursquare_data.utcTimestamp.dt.round('60min')
new_df = pd.merge(foursquare_data, climate_data,  how='left', left_on=['closestStation','roundedDate'], right_on = ['STATION_NAME','roundedDate'])
new_df.sample(10)

,userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp,lat_long_x,closestStation,rountedDate,roundedDate,STATION,STATION_NAME,ELEVATION,LATITUDE,LONGITUDE,DATE,REPORTTPYE,HOURLYSKYCONDITIONS,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,HOURLYDRYBULBTEMPF,HOURLYDRYBULBTEMPC,HOURLYWETBULBTEMPF,HOURLYWETBULBTEMPC,HOURLYDewPointTempF,HOURLYDewPointTempC,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYWindGustSpeed,HOURLYStationPressure,HOURLYPressureTendency,HOURLYPressureChange,HOURLYSeaLevelPressure,HOURLYPrecip,HOURLYAltimeterSetting,DAILYMaximumDryBulbTemp,DAILYMinimumDryBulbTemp,DAILYAverageDryBulbTemp,DAILYDeptFromNormalAverageTemp,DAILYAverageRelativeHumidity,DAILYAverageDewPointTemp,DAILYAverageWetBulbTemp,DAILYHeatingDegreeDays,DAILYCoolingDegreeDays,DAILYSunrise,DAILYSunset,DAILYWeather,DAILYPrecip,DAILYSnowfall,DAILYSnowDepth,DAILYAverageStationPressure,DAILYAverageSeaLevelPressure,DAILYAverageWindSpeed,DAILYPeakWindSpeed,PeakWindDirection,DAILYSustainedWindSpeed,DAILYSustainedWindDirection,MonthlyMaximumTemp,MonthlyMinimumTemp,MonthlyMeanTemp,MonthlyAverageRH,MonthlyDewpointTemp,MonthlyWetBulbTemp,MonthlyAvgHeatingDegreeDays,MonthlyAvgCoolingDegreeDays,MonthlyStationPressure,MonthlySeaLevelPressure,MonthlyAverageWindSpeed,MonthlyTotalSnowfall,MonthlyDeptFromNormalMaximumTemp,MonthlyDeptFromNormalMinimumTemp,MonthlyDeptFromNormalAverageTemp,MonthlyDeptFromNormalPrecip,MonthlyTotalLiquidPrecip,MonthlyGreatestPrecip,MonthlyGreatestPrecipDate,MonthlyGreatestSnowfall,MonthlyGreatestSnowfallDate,MonthlyGreatestSnowDepth,MonthlyGreatestSnowDepthDate,MonthlyDaysWithGT90Temp,MonthlyDaysWithLT32Temp,MonthlyDaysWithGT32Temp,MonthlyDaysWithLT0Temp,MonthlyDaysWithGT001Precip,MonthlyDaysWithGT010Precip,MonthlyDaysWithGT1Snow,MonthlyMaxSeaLevelPressureValue,MonthlyMaxSeaLevelPressureDate,MonthlyMaxSeaLevelPressureTime,MonthlyMinSeaLevelPressureValue,MonthlyMinSeaLevelPressureDate,MonthlyMinSeaLevelPressureTime,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD,lat_long_y
277392,860,4deffce2b0fbfee72f3cacde,4d4b7105d754a06374d81259,Food,40.804742,-73.955439,-300,2012-11-24 01:25:09,"(40.80474202, -73.95543871)",JFK INTERNATIONAL AIRPORT NY US,2012-11-24 01:00:00,2012-11-24 01:00:00,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2012-11-24 00:51:00,FM-15,BKN:07 80 OVC:08 95,10.00,NaN,48,8.9,46.0,7.8,44,6.7,86.0,8.0,260,NaN,29.67,6.0,NaN,29.69,0.01,29.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,653.0,1631.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999.0,-9999.0,NaN,-9999.0,-9999.0,NaN,NaN,NaN,NaN,NaN,NaN,"(40.6386, -73.7622)"
228719,521,4b15b66bf964a52015b323e3,4bf58dd8d48988d176941735,Gym / Fitness Center,40.753727,-73.977342,-240,2012-08-17 17:14:31,"(40.75372664, -73.97734165)",JFK INTERNATIONAL AIRPORT NY US,2012-08-17 17:00:00,2012-08-17 17:00:00,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2012-08-17 16:51:00,FM-15,FEW:02 50 SCT:04 150 SCT:04 250,9.00,NaN,81,27.2,74.0,23.4,71,21.7,72.0,17.0,190,NaN,29.77,NaN,NaN,29.79,0.00,29.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,508.0,1849.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999.0,-9999.0,NaN,-9999.0,-9999.0,NaN,NaN,NaN,NaN,NaN,NaN,"(40.6386, -73.7622)"
31320,496,4b676520f964a520404c2be3,4bf58dd8d48988d176941735,Gym / Fitness Center,40.849745,-73.934894,-240,2012-04-19 22:51:21,"(40.8497448, -73.93489428)",JFK INTERNATIONAL AIRPORT NY US,2012-04-19 23:00:00,2012-04-19 23:00:00,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2012-04-19 22:51:00,FM-15,NaN,10.00,NaN,51,10.6,49.0,9.7,48,8.9,89.0,6.0,190,NaN

In [107]:
new_df[new_df.venueId == '49c54c2cf964a520ee561fe3']
new_df2 = pd.merge(foursquare_data, climate_data,  how='inner', left_on=['closestStation','roundedDate'], right_on = ['STATION_NAME','roundedDate'])
new_df2.sample(5)
new_df2[new_df2.venueId == '3fd66200f964a52021e41ee3']

,userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp,lat_long_x,closestStation,rountedDate,roundedDate,STATION,STATION_NAME,ELEVATION,LATITUDE,LONGITUDE,DATE,REPORTTPYE,HOURLYSKYCONDITIONS,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,HOURLYDRYBULBTEMPF,HOURLYDRYBULBTEMPC,HOURLYWETBULBTEMPF,HOURLYWETBULBTEMPC,HOURLYDewPointTempF,HOURLYDewPointTempC,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYWindGustSpeed,HOURLYStationPressure,HOURLYPressureTendency,HOURLYPressureChange,HOURLYSeaLevelPressure,HOURLYPrecip,HOURLYAltimeterSetting,DAILYMaximumDryBulbTemp,DAILYMinimumDryBulbTemp,DAILYAverageDryBulbTemp,DAILYDeptFromNormalAverageTemp,DAILYAverageRelativeHumidity,DAILYAverageDewPointTemp,DAILYAverageWetBulbTemp,DAILYHeatingDegreeDays,DAILYCoolingDegreeDays,DAILYSunrise,DAILYSunset,DAILYWeather,DAILYPrecip,DAILYSnowfall,DAILYSnowDepth,DAILYAverageStationPressure,DAILYAverageSeaLevelPressure,DAILYAverageWindSpeed,DAILYPeakWindSpeed,PeakWindDirection,DAILYSustainedWindSpeed,DAILYSustainedWindDirection,MonthlyMaximumTemp,MonthlyMinimumTemp,MonthlyMeanTemp,MonthlyAverageRH,MonthlyDewpointTemp,MonthlyWetBulbTemp,MonthlyAvgHeatingDegreeDays,MonthlyAvgCoolingDegreeDays,MonthlyStationPressure,MonthlySeaLevelPressure,MonthlyAverageWindSpeed,MonthlyTotalSnowfall,MonthlyDeptFromNormalMaximumTemp,MonthlyDeptFromNormalMinimumTemp,MonthlyDeptFromNormalAverageTemp,MonthlyDeptFromNormalPrecip,MonthlyTotalLiquidPrecip,MonthlyGreatestPrecip,MonthlyGreatestPrecipDate,MonthlyGreatestSnowfall,MonthlyGreatestSnowfallDate,MonthlyGreatestSnowDepth,MonthlyGreatestSnowDepthDate,MonthlyDaysWithGT90Temp,MonthlyDaysWithLT32Temp,MonthlyDaysWithGT32Temp,MonthlyDaysWithLT0Temp,MonthlyDaysWithGT001Precip,MonthlyDaysWithGT010Precip,MonthlyDaysWithGT1Snow,MonthlyMaxSeaLevelPressureValue,MonthlyMaxSeaLevelPressureDate,MonthlyMaxSeaLevelPressureTime,MonthlyMinSeaLevelPressureValue,MonthlyMinSeaLevelPressureDate,MonthlyMinSeaLevelPressureTime,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD,lat_long_y
47250,375,3fd66200f964a52021e41ee3,4bf58dd8d48988d116941735,Bar,40.719854,-73.988492,-240,2012-04-28 21:31:49,"(40.71985365, -73.98849235)",JFK INTERNATIONAL AIRPORT NY US,2012-04-28 22:00:00,2012-04-28 22:00:00,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2012-04-28 21:51:00,FM-15,OVC:08 120,10.00,NaN,53,11.7,41.0,5.3,25,-3.9,34.0,11.0,280,NaN,30.12,1.0,NaN,30.14,0.00,30.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,455,1850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN,"(40.6386, -73.7622)"
47251,375,3fd66200f964a52021e41ee3,4bf58dd8d48988d116941735,Bar,40.719854,-73.988492,-240,2012-04-28 21:31:49,"(40.71985365, -73.98849235)",JFK INTERNATIONAL AIRPORT NY US,2012-04-28 22:00:00,2012-04-28 22:00:00,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2012-04-28 22:00:00,FM-12,NaN,9.94,NaN,53,11.7,41.0,5.3,25,-3.9,34.0,11.0,280,NaN,30.11,1.0,-0.03,30.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,455,1850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN,"(40.6386, -73.7622)"
48369,122,3fd66200f964a52021e41ee3,4bf58dd8d48988d116941735,Bar,40.719854,-73.988492,-240,2012-04-29 03:07:26,"(40.71985365, -73.98849235)",JFK INTERNATIONAL AIRPORT NY US,2012-04-29 03:00:00,2012-04-29 03:00:00,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2012-04-29 02:51:00,FM-15,NaN,10.00,NaN,47,8.3,40.0,4.2,29,-1.7,50.0,3.0,330,NaN,30.10,NaN,NaN,30.12,0.00,30.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,455,1850,NaN,NaN,NaN,NaN,NaN